In [22]:
import pickle
import re
import numpy as np

basis_file = '/home/anton/Documents/Tulane/Research/PNC_Good/AngleBasisLong5.pkl'
demodir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/PNC/'

basis = pickle.load(open(basis_file, 'rb'))

pnc_thetas = []
pnc_jitter = []
pnc_sex = []
pnc_race = []
pnc_age = []
pnc_fc = []

demo = pickle.load(open(f'{demodir}/demographics.pkl', 'rb'))
pnc_subs = []
pnc_tasks = []

for subtask in basis:
    m = re.search('([^-]+)-(.*)', subtask)
    sub = m.group(1)
    task = m.group(2)
    s = demo['Sex'][sub] == 'M'
    r = demo['Race'][sub]
    a = demo['age_at_cnb'][sub]
    if r not in ['AA', 'EA']:
        continue
    pnc_subs.append(sub)
    pnc_tasks.append(task)
    pnc_sex.append(s)
    pnc_race.append(r == 'AA')
    pnc_age.append(a)
    pnc_thetas.append(basis[subtask]['thetas'])
    pnc_jitter.append(basis[subtask]['jitter'])
    p = np.load(f'{demodir}/fc/{sub}_task-{task}_fc.npy')
    pnc_fc.append(p)
    
pnc_thetas = np.stack(pnc_thetas)
pnc_jitter = np.stack(pnc_jitter)
pnc_sex = np.array(pnc_sex).astype('int')
pnc_race = np.array(pnc_race).astype('int')
pnc_age = np.array(pnc_age).astype('int')
pnc_fc = np.stack(pnc_fc)

print([a.shape for a in [pnc_thetas, pnc_jitter, pnc_sex, pnc_race, pnc_fc, pnc_age]])

[(3849, 5, 264), (3849, 5, 264), (3849,), (3849,), (3849, 34716), (3849,)]


In [6]:
import pickle
import re
import numpy as np

basis_file = '/home/anton/Documents/Tulane/Research/BSNIP/AngleBasisLong5.pkl'
demodir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/BSNIP/'

basis = pickle.load(open(basis_file, 'rb'))

bsnip_thetas = []
bsnip_jitter = []
bsnip_fc = []
bsnip_race = []
bsnip_sex = []

demo = pickle.load(open(f'{demodir}/demographics.pkl', 'rb'))
bsnip_subs = []

for subtask in basis:
    m = re.search('([^-]+)', subtask)
    sub = m.group(1)
    s = demo['sex'][sub] == 's1.0' # Male
    r = demo['Race'][sub]
    if r not in ['AA', 'CA']:
        continue
    bsnip_sex.append(s)
    bsnip_race.append(r == 'AA')
    bsnip_subs.append(sub)
    bsnip_thetas.append(basis[subtask]['thetas'])
    bsnip_jitter.append(basis[subtask]['jitter'])
    p = np.load(f'{demodir}/fc/{sub}_task-unk_fc.npy')
    bsnip_fc.append(p)
    
bsnip_thetas = np.stack(bsnip_thetas)
bsnip_jitter = np.stack(bsnip_jitter)
bsnip_fc = np.stack(bsnip_fc)
bsnip_sex = np.array(bsnip_sex).astype('int')
bsnip_race = np.array(bsnip_race).astype('int')

print([a.shape for a in [bsnip_thetas, bsnip_jitter, bsnip_fc, bsnip_sex, bsnip_race]])

[(1165, 5, 264), (1165, 5, 264), (1165, 34716), (1165,), (1165,)]


In [3]:
def rmse(yhat, y):
    if isinstance(yhat, np.ndarray) or isinstance(yhat, int):
        f = np.mean
    else:
        f = torch.mean
    return f((y-yhat)**2)**0.5

def tops(thetas, jitter):
    t0 = np.expand_dims(thetas, 2)
    t1 = np.expand_dims(thetas, 3)
    j0 = np.expand_dims(jitter, 2)
    j1 = np.expand_dims(jitter, 3)
    ps = np.cos(t0-t1)*(j0*j1)
    a,b = np.triu_indices(264, 1)
    ps = ps[:,:,a,b]
    return ps

print('Complete')

Complete


In [4]:
# AB1

ps = tops(pnc_thetas, pnc_jitter)
pnc_ab1 = np.mean(ps, axis=1)
pnc_res1 = pnc_fc - pnc_ab1

print(pnc_ab1.shape)
print(pnc_res1.shape)

ps = tops(bsnip_thetas, bsnip_jitter)
bsnip_ab1 = np.mean(ps, axis=1)
bsnip_res1 = bsnip_fc - bsnip_ab1

print(bsnip_ab1.shape)
print(bsnip_res1.shape)

(3849, 34716)
(3849, 34716)
(1165, 34716)
(1165, 34716)


In [7]:
# AB5

ps = tops(pnc_thetas, pnc_jitter)
pnc_ab5 = np.mean(ps, axis=1)
pnc_res5 = pnc_fc - pnc_ab5

print(pnc_ab5.shape)
print(pnc_res5.shape)

ps = tops(bsnip_thetas, bsnip_jitter)
bsnip_ab5 = np.mean(ps, axis=1)
bsnip_res5 = bsnip_fc - bsnip_ab5

print(bsnip_ab5.shape)
print(bsnip_res5.shape)

(3849, 34716)
(3849, 34716)
(1165, 34716)
(1165, 34716)


In [26]:
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

def cat(x, **kwargs):
    return np.concatenate(x, **kwargs)

wdir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/PNC/weights/'

def get_next_idx(model, inp):
    mx = -1
    for f in Path(f'{wdir}/{inp}/').iterdir():
        m = re.match(f'{model}([\d]+).pkl', f.name)
        if m:
            idx = int(m.group(1))
            if idx > mx:
                mx = idx
    return mx+1

def save(model, inp, rmse, trsubs, tsubs, w):
    num = get_next_idx(model, inp)
    desc = f'{model} {inp} rmse: {rmse}'
    dct = dict(w=w.squeeze(), trsubs=trsubs, tsubs=tsubs, desc=desc)
    with open(f'{wdir}/{inp}/{model}{num}.pkl', 'wb') as f:
        pickle.dump(dct, f)

sel = ((1-pnc_sex)*(1-pnc_race)).astype('bool')
x = pnc_res1[sel]
y = pnc_age[sel]
idcs = np.arange(x.shape[0])

xtr, xt, ytr, yt, tridcs, tidcs = train_test_split(x, y, idcs, train_size=0.8)
reg = Ridge(alpha=1).fit(xtr, ytr) #LogisticRegression(C=1, max_iter=1000).fit(xtr, ytr)
yhat = reg.predict(xt)
# p = reg.predict_proba(xt)
# auc = roc_auc_score(yt, p[:,1])
# print(np.mean(yhat == yt))
auc = rmse(yt, yhat)
print(auc)

save('ridge', 
     'res1-eaf-age', 
     auc, 
     [pnc_subs[i] for i in tridcs], 
     [pnc_subs[i] for i in tidcs], 
     reg.coef_.squeeze())

save('ridge-wf',
    'res1-eaf-age', 
     auc, 
     [pnc_subs[i] for i in tridcs], 
     [pnc_subs[i] for i in tidcs], 
     reg.coef_.squeeze()*np.mean(xtr, axis=0))

1.8607776679356014


In [11]:
reg.coef_.squeeze()

array([ 0.044449  ,  0.03763571,  0.02345932, ..., -0.01904058,
        0.03514169,  0.00912824])